In [2]:
import pandas as pd

# Ler o arquivo Excel
df = pd.read_excel('test_5.xlsx')

# Exibir os dados para verificar
print(df.head())

     Root     Branch      Sub-Branch      Leaf                  URL
0  Root 1  Branch 1A  Sub-Branch 1A1  Leaf 1A1  https://link1a1.com
1  Root 1  Branch 1A  Sub-Branch 1A2  Leaf 1A2  https://link1a2.com
2  Root 1  Branch 1B  Sub-Branch 1B1  Leaf 1B1  https://link1b1.com
3  Root 2  Branch 2A  Sub-Branch 2A1  Leaf 2A1  https://link2a1.com
4  Root 3  Branch 3A  Sub-Branch 3A1  Leaf 3A1  https://link3a1.com


In [3]:
class Node:
    def __init__(self, name, url=None):
        self.name = name
        self.url = url
        self.children = []

    def add_child(self, child_node):
        self.children.append(child_node)

    def __repr__(self, level=0):
        ret = "\t" * level + repr(self.name) + "\n"
        for child in self.children:
            ret += child.__repr__(level + 1)
        return ret

In [4]:
class Forest:
    def __init__(self):
        self.roots = []

    def add_tree(self, root_node):
        self.roots.append(root_node)

    def __repr__(self):
        ret = ""
        for root in self.roots:
            ret += repr(root) + "\n"
        return ret

def build_forest_from_df(df):
    forest = Forest()
    root_map = {}
    
    for _, row in df.iterrows():
        root_name = row['Root']
        branch_name = row['Branch']
        sub_branch_name = row['Sub-Branch']
        leaf_name = row['Leaf']
        url = row['URL']

        if root_name not in root_map:
            root_node = Node(root_name)
            forest.add_tree(root_node)
            root_map[root_name] = root_node
        else:
            root_node = root_map[root_name]
        
        branch_node = next((child for child in root_node.children if child.name == branch_name), None)
        if not branch_node:
            branch_node = Node(branch_name)
            root_node.add_child(branch_node)

        sub_branch_node = next((child for child in branch_node.children if child.name == sub_branch_name), None)
        if not sub_branch_node:
            sub_branch_node = Node(sub_branch_name)
            branch_node.add_child(sub_branch_node)

        leaf_node = next((child for child in sub_branch_node.children if child.name == leaf_name), None)
        if not leaf_node:
            leaf_node = Node(leaf_name, url)
            sub_branch_node.add_child(leaf_node)

    return forest


In [5]:
def generate_html(node):
    if not node.children:
        return f'<li>{node.name}</li>'
    children_html = ''.join(generate_html(child) for child in node.children)
    return f'''
    <li>
        <span class="caret">{node.name}</span>
        <ul class="nested">
            {children_html}
        </ul>
    </li>
    '''

def generate_forest_html(forest):
    forest_html = ''.join(generate_html(root) for root in forest.roots)
    return f'''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Collapsible Tree Menu</title>
    <style>
        ul {{
            list-style-type: none; /* Remove bullet points */
        }}
        .nested {{
            display: none;
        }}
        .active {{
            display: block;
        }}
        .caret {{
            cursor: pointer;
            user-select: none;
            font-weight: bold; /* Highlight caret items */
            color: #006c66; /* Replace with the color of MPIL logo */
        }}
    </style>
</head>
<body>
    <h2>Collapsible Tree Menu</h2>
    <ul id="myMenu">
        {forest_html}
    </ul>
    <script>
        document.addEventListener('DOMContentLoaded', function() {{
            var toggler = document.getElementsByClassName("caret");
            for (var i = 0; i < toggler.length; i++) {{
                toggler[i].addEventListener("click", function() {{
                    this.parentElement.querySelector(".nested").classList.toggle("active");
                }});
            }}
        }});
    </script>
</body>
</html>
    '''


In [6]:
# Ler o arquivo Excel
df = pd.read_excel('test_5.xlsx')

# Construir a floresta a partir do DataFrame
forest = build_forest_from_df(df)

# Gerar HTML
html_content = generate_forest_html(forest)

# Salvar o HTML em um arquivo
with open('Test_5.html', 'w') as file:
    file.write(html_content)

print("HTML file 'Test_5.html' has been generated.")


HTML file 'Test_5.html' has been generated.
